### Arrange Data

In [1]:
import mysql.connector
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def load_properties(filepath, sep='=', comment_char='#'):
    """
    Read the file passed as parameter as a properties file.
    """
    props = {}
    with open(filepath, "rt") as f:
        for line in f:
            l = line.strip()
            if l and not l.startswith(comment_char):
                key_value = l.split(sep)
                key = key_value[0].strip()
                value = sep.join(key_value[1:]).strip().strip('"') 
                props[key] = value 
    return props

In [3]:
connection = mysql.connector.connect(**load_properties('./project.properties'))

In [ ]:
query = '''
select *, avg(rainfall) OVER(ORDER BY date
     ROWS BETWEEN 2 PRECEDING AND current row)
     as moving_average from weatherdailydelay where sbbregion_isocode = 'RWT'
'''

result = pd.read_sql(sql=query, con=connection)
result_df = pd.DataFrame(result)

C:\Users\micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
result_df.head()

In [ ]:
### change puenktlichkeit to delay, for better visual understanding of correlation to rain and temperature

result_df.zugpuenktlichkeit = 100 - result_df.zugpuenktlichkeit

In [ ]:
result_df.head()

In [ ]:
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px

### Below Zero

In [ ]:
filtered_df = result_df[result_df['temp'] < 0]

In [ ]:
max_rainfall = max(filtered_df['rainfall'])

In [ ]:
rain_range = np.arange(0, max_rainfall + max_rainfall / 5, max_rainfall / 5)

In [ ]:
grouped_df = filtered_df.groupby(pd.cut(filtered_df['rainfall'], rain_range)).mean()

In [ ]:
grouped_df

In [ ]:
text_array = [(str(round(value, 1)) + "% der Züge haben mindestens 3 Min Verspätung") for value in grouped_df['zugpuenktlichkeit'].to_numpy()]

### Lineplot

In [ ]:
datatrace1 = {
    'name': 'Rainfall',
    'type': 'bar', 
    'y': rain_range, 
    'x': grouped_df['zugpuenktlichkeit'],
    'orientation': 'h',
    'text': text_array,
    'marker':{'color':'#dba74b'}
}

layout = {
    'title': 'SBB Verspätung gruppiert nach Schneefall'
}

figdict = {'data': [datatrace1], 
          'layout': layout}

fig = go.Figure(**figdict)

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = rain_range,
        ticktext = ['Wenig bis kein Schneefall','Wenig Schneefall', 'Mittlerer Schneefall','Mittelstarker Schneefall','Starker Schneefall','']
    )
)

fig.update_traces(textposition='auto')
fig.show()

In [ ]:
fig = px.bar

### Above Zero

In [ ]:
filtered_above_zero_df = result_df[result_df['temp'] > 0]

In [ ]:
max_rainfall_above_zero = max(filtered_above_zero_df['rainfall'])

In [ ]:
rain_range_above_zero = np.arange(0, max_rainfall_above_zero + max_rainfall_above_zero / 3, max_rainfall_above_zero / 3)

In [ ]:
grouped_above_zero_df = filtered_above_zero_df.groupby(pd.cut(filtered_above_zero_df['rainfall'], rain_range)).mean()

In [ ]:
grouped_above_zero_df

### Lineplot

In [ ]:
datatrace1 = {
    'name': 'Delay when temperature above 0',
    'type': 'bar', 
    'x': rain_range, 
    'y': grouped_above_zero_df['zugpuenktlichkeit']
}
datatrace2 = {
    'name': 'Delay when temperature below 0',
    'type': 'bar', 
    'x': rain_range, 
    'y': grouped_df['zugpuenktlichkeit'],
}


layout = {
    'title': 'SBB Verspätung gruppiert nach Regenmenge'
}

figdict = {'data': [datatrace1, datatrace2], 
          'layout': layout}

go.Figure(**figdict)